## Imports

In [1]:
import pandas as pd
import numpy as np
import warnings
from helper_functions.file_io_functions import detect_raw_files, load_processed_data, save_results, load_data, create_excel_files_from_nested_dict
from helper_functions.cleanup_functions import raw_cleanup
from helper_functions.edges_clusters import find_name_adresse_doubletten
from helper_functions.filter_muster_organisationen import general_exclusion_criteria, FDA_servicerole, batch_process_produkte, organisationsrollen_filter_and_format_batch, find_portal_vs_physisch_doublette, find_frequent_roles
from helper_functions.analyses_formatting import final_touch, final_touch_batch, add_organisationsrollen_string_columns, organisationsrollen_add_inhaber_typ_and_produkt_typ
from helper_functions.statistics import count_produktrollen_identische_sonstige
import pickle

# This extension will cause imported modules to be reloaded if there were changes made.
%load_ext autoreload
%autoreload 2

# Disable some annoying warnings. 
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

# Set the global number formatting
np.set_printoptions(precision=2, suppress=True)

## Loading Data & Basic Cleanup

Specify data folder. Can be the data folder of GraphViewer App.

This step must only be run once when processing a new dataset.

Recommendation:
When loading new data into the GraphViewerApp, run processing first there. 
Then run this script with "skip_hyperlink_step=True" below (a time consuming step that extracts links the formatted excel files).

In [2]:
# To check if all required Expertensuche files exist.
raw_data_directory = "../GraphViewerApp/data/"

raw_files, error_message = detect_raw_files(directory=raw_data_directory)
if error_message:
    print(error_message)

In [3]:
df_organisationen, df_personen = raw_cleanup(raw_files, raw_data_directory, skip_hyperlink_step=True)  # Takes >5 min

Reading excel files and extracting hyperlinks (takes several minutes)...
Basic cleanup Organisationen & Personen...
Aggregating additional Expertensuchen...
Storing dataframes as pickle...


In [4]:
# df_organisationen.to_excel('output/Organisationen_processed.xlsx', engine='openpyxl', index=False)

### Optionally, skip above processing and load cleaned data

In [5]:
data_dfs = load_processed_data()

df_organisationen = data_dfs["organisationen"]
df_personen = data_dfs["personen"]
df_organisationsrollen = data_dfs["organisationsrollen"]

In [6]:
# For newer analysis, enhance Organisationsrollen file with Produkttyp / Inhabertyp
df_organisationsrollen = organisationsrollen_add_inhaber_typ_and_produkt_typ(df_organisationsrollen, df_personen, df_organisationen)
df_organisationsrollen.to_excel('data/calculated/Organisationsrollen_processed.xlsx', engine='openpyxl', index=False)

Number of rows with a mixture of 'Person' and 'Organisation': 249


## Performance Analysis
Simply checks which Organisationen OR Personen have >1000 roles as Inhaber / Rechnungsempfänger / Korrespondenzempfänger

In [7]:
inhaber_df, rechnungsempfaenger_df, korrespondenzempfaenger_df = find_frequent_roles(df_organisationsrollen)

with pd.ExcelWriter('output/Organisationen_Frequent_Roles.xlsx', engine='openpyxl') as writer:
    inhaber_df.to_excel(writer, sheet_name='Inhaber', index=False)
    rechnungsempfaenger_df.to_excel(writer, sheet_name='Rechnungsempfaenger', index=False)
    korrespondenzempfaenger_df.to_excel(writer, sheet_name='Korrespondenzempfaenger', index=False)

## Find Doubletten

In [8]:
# Starting point for most analyses: Have groups of organisations with exact match in Name and Adresse.

organisationen_doubletten = find_name_adresse_doubletten(df_organisationen)
doubletten_list_organisationen = df_organisationen["ReferenceID"].unique().tolist()

In [9]:
# ----- Some general exclusion criteria: ------

no_Produkte = False
no_Geschaeftspartner = False # So fare we only analyzed without Geschäftspartner
no_Servicerole = False # So far we only analyzed without Servicerole

# For analyzing data with BAZL, BAFU etc.  (must set no_Geschaefstpartner to False above)
only_with_Geschaeftspartner = True

organisationen_doubletten_filtered = general_exclusion_criteria(organisationen_doubletten, no_Produkte=no_Produkte, no_Geschaeftspartner=no_Geschaeftspartner, no_Servicerole=no_Servicerole, only_with_Geschaeftspartner=only_with_Geschaeftspartner)


### Organisationsrollenanalyse

Create analysis for single product or list of products, e.g "Einzelnummer". Find doubletten that have roles related to that product.

Takes >20 min for all products.

Choices are:
`
'116xyz-Kurznummer',
'18xy-Kurznummer',
'1xy-Kurznummer',
'Carrier Selection Code (CSC)', 
'E.164-Nummernblock', 
'E.164-Zugangskennzahl', 
'Einzelnummer', 
'International Signalling Point Code (ISPC)', 
'Issuer Identifier Number (IIN)', 
'Mobile Network Code (MNC)', 
'National Signalling Point Code (NSPC)', 
'Objektbezeichner (OID)', 
'Weiteres Adressierungselement', 
'Packet Radio Rufzeichen', 
'Rufzeichen Amateurfunk', 
'Rufzeichen Hochseeyacht', 
'Rufzeichen Luftfahrzeug', 
'Rufzeichen Rheinschiff', 
'Rufzeichen SOLAS-Schiff', 
'Handsprechfunkgeräte mit DSC (Maritime Kennung)', 
'FDA'
`


In [10]:
produktnamen = ['116xyz-Kurznummer',
'18xy-Kurznummer',
'1xy-Kurznummer',
'Carrier Selection Code (CSC)', 
'E.164-Nummernblock', 
'E.164-Zugangskennzahl', 
'Einzelnummer', 
'International Signalling Point Code (ISPC)', 
'Issuer Identifier Number (IIN)', 
'Mobile Network Code (MNC)', 
'National Signalling Point Code (NSPC)', 
'Objektbezeichner (OID)', 
'Weiteres Adressierungselement', 
'Packet Radio Rufzeichen', 
'Rufzeichen Amateurfunk', 
'Rufzeichen Hochseeyacht', 
'Rufzeichen Luftfahrzeug', 
'Rufzeichen Rheinschiff', 
'Rufzeichen SOLAS-Schiff', 
'Handsprechfunkgeräte mit DSC (Maritime Kennung)', 
'FDA']

# produktnamen = ["E.164-Nummernblock"]

organisationsrollen_results_3_roles, organisationsrollen_results_2_roles = batch_process_produkte(organisationen_doubletten_filtered, df_organisationsrollen, produktnamen) 

✅ Done with 116xyz-Kurznummer
❌ No Doubletten with 3 roles found!
❌ No Doubletten with 2 roles found!
✅ Done with 18xy-Kurznummer
❌ No Doubletten with 3 roles found!
❌ No Doubletten with 2 roles found!
✅ Done with 1xy-Kurznummer
❌ No Doubletten with 2 roles found!
✅ Done with Carrier Selection Code (CSC)
❌ No Doubletten with 3 roles found!
❌ No Doubletten with 2 roles found!
✅ Done with E.164-Nummernblock
❌ No Doubletten with 3 roles found!
❌ No Doubletten with 2 roles found!
✅ Done with E.164-Zugangskennzahl
❌ No Doubletten with 3 roles found!
❌ No Doubletten with 2 roles found!
✅ Done with Einzelnummer
✅ Done with International Signalling Point Code (ISPC)
❌ No Doubletten with 3 roles found!
❌ No Doubletten with 2 roles found!
✅ Done with Issuer Identifier Number (IIN)
❌ No Doubletten with 3 roles found!
❌ No Doubletten with 2 roles found!
✅ Done with Mobile Network Code (MNC)
❌ No Doubletten with 3 roles found!
❌ No Doubletten with 2 roles found!
✅ Done with National Signalling Poin

In [11]:
# To skip above steps, store and re-load results like this:
save_results(organisationsrollen_results_3_roles, "organisationsrollen_3_roles.pickle")
save_results(organisationsrollen_results_2_roles, "organisationsrollen_2_roles.pickle")
organisationsrollen_results_2_roles = load_data("data/calculated/organisationsrollen_2_roles.pickle") 
organisationsrollen_results_3_roles = load_data("data/calculated/organisationsrollen_3_roles.pickle") 

Erklärung zu den Parametern:  

*roles_per_product*: Produkte haben 3 Rollen (Inhaber, Rechempf., Korrempf.). Für gewisse Analysen wollen wir, dass zwei der drei Rollen in einer Gruppe Doubletten vorhanden sind, für andere wollen wir alle drei.

*rows_per_product*: Für den Fall, dass alle drei Rollen in den Doubletten vorkommen, können diese auf zwei Doubletten (rows) aufgeteilt sein, oder auf drei (Fall "komplette Doublette").

In [12]:
# takes ~5min

organisationsrollen_results_formatted_2, s_df1 = organisationsrollen_filter_and_format_batch(organisationsrollen_results_2_roles, roles_per_product=2)
organisationsrollen_results_formatted_komplette_doublette, s_df2 = organisationsrollen_filter_and_format_batch(organisationsrollen_results_3_roles, rows_per_product=3, roles_per_product=3)
organisationsrollen_results_formatted_3, s_df3 = organisationsrollen_filter_and_format_batch(organisationsrollen_results_3_roles, rows_per_product=2, roles_per_product=3)

In [13]:
# combine statistics dfs
combined_df = pd.concat([s_df1, s_df2, s_df3])
organisationsrollen_statistik_df = combined_df.groupby('produkte', as_index=False)['Doubletten'].sum()


In [14]:
cols_to_keep=["ReferenceID", "Name", "Objekt_link", "address_full", "VerknuepftesObjekt_list", "VerknuepftesObjektID_list", "Geschaeftspartner", "cluster_id", "score", "master", "masterID", "Inhaber_Objekt", "Rechempf_Objekt", "Korrempf_Objekt", "Inhaber_ProduktID", "Rechempf_ProduktID", "Korrempf_ProduktID"]
organisationsrollen_results_formatted_2 = final_touch_batch(organisationsrollen_results_formatted_2, cols_to_keep, two_roles=True)
organisationsrollen_results_formatted_3 = final_touch_batch(organisationsrollen_results_formatted_3, cols_to_keep, alphanumeric=True)
organisationsrollen_results_formatted_komplette_doublette = final_touch_batch(organisationsrollen_results_formatted_komplette_doublette, cols_to_keep, alphanumeric=True)


In [15]:
# Create excel file for each Produkt-type in /output folder.  (Error message "ValueError: seek of closed file" is expected and can be ignored)

create_excel_files_from_nested_dict(organisationsrollen_results_formatted_2, output_dir='output/2_organisationsrollen') 
create_excel_files_from_nested_dict(organisationsrollen_results_formatted_3, output_dir='output/3_organisationsrollen')
create_excel_files_from_nested_dict(organisationsrollen_results_formatted_komplette_doublette, output_dir='output/organisationsrollen_komplette_doublette')

No sheets created for file 'Rufzeichen Luftfahrzeug' as all dataframes are empty.
No sheets created for file '1xy-Kurznummer' as all dataframes are empty.
No sheets created for file 'Rufzeichen Luftfahrzeug' as all dataframes are empty.


### Statistik
Mengengerüst an Doubletten mit sektionsübergreifenden Produkttypen.
Doubletten, die eine Rolle haben zum selben Produkt mit Typ Funk.

In [16]:
statistik_df = count_produktrollen_identische_sonstige(df_organisationsrollen, organisationsrollen_statistik_df)
statistik_df.to_excel('output/Statistik_Produkte_Rollen.xlsx', engine='openpyxl', index=False)

### FDA Servicerollenanalyse

In [19]:
len(df_FDA_servicerole)

0

In [17]:
if only_with_Geschaeftspartner:
    organisationen_doubletten_filtered = general_exclusion_criteria(organisationen_doubletten, no_Produkte=True, no_Geschaeftspartner=False, no_Servicerole=False, only_with_Geschaeftspartner=True)
    cols_to_keep=["ReferenceID", "Name", "Objekt_link", "address_full", "VerknuepftesObjekt_list", "VerknuepftesObjektID_list", "Geschaeftspartner", "Servicerole_string", "cluster_id", "score", "master", "masterID"]
else:
    organisationen_doubletten_filtered = general_exclusion_criteria(organisationen_doubletten, no_Produkte=True, no_Geschaeftspartner=True, no_Servicerole=False)
    cols_to_keep=["ReferenceID", "Name", "Objekt_link", "address_full", "VerknuepftesObjekt_list", "VerknuepftesObjektID_list", "Servicerole_string", "cluster_id", "score", "master", "masterID"]

df_FDA_servicerole = FDA_servicerole(organisationen_doubletten_filtered)

if len(df_FDA_servicerole) > 0:
    df_FDA_servicerole = final_touch(df_FDA_servicerole, cols_to_keep)
    df_FDA_servicerole.to_excel('output/Organisationen_FDA_Servicerole.xlsx', engine='openpyxl', index=False)

ValueError: DataFrame does not contain 'cluster_id' column.

### Doubletten Ohne Produkte

In [20]:
if only_with_Geschaeftspartner:
    organisationen_doubletten_filtered = general_exclusion_criteria(organisationen_doubletten, no_Produkte=True, no_Geschaeftspartner=False, no_Servicerole=True, only_with_Geschaeftspartner=True)
    cols_to_keep=["ReferenceID", "Name_Zeile2", "Objekt_link", "address_full", "VerknuepftesObjekt_list", "VerknuepftesObjektID_list", "Geschaeftspartner", "cluster_id", "score", "master", "masterID"]
else:
    organisationen_doubletten_filtered = general_exclusion_criteria(organisationen_doubletten, no_Produkte=True, no_Geschaeftspartner=True, no_Servicerole=True)
    cols_to_keep=["ReferenceID", "Name_Zeile2", "Objekt_link", "address_full", "VerknuepftesObjekt_list", "VerknuepftesObjektID_list", "cluster_id", "score", "master", "masterID"]

df_doubletten_ohne_produkte = final_touch(organisationen_doubletten_filtered, cols_to_keep)

df_doubletten_ohne_produkte.to_excel('output/Organisationen_ohne_produkte.xlsx', engine='openpyxl', index=False)

### Physisch vs. Portal

Any Doubletten that have same Name + Address + Email (though one may have empty email), irrespective of Organisationsrollen (but those are in a column as list)

Currently Geschaeftspartner / Servicerolle are allowed, and shown in separate columns.

In [21]:
df_portal_physisch_only_nonempty_email = find_portal_vs_physisch_doublette(df_organisationen, organisationen=True, strict_email=True, only_Geschaeftspartner=only_with_Geschaeftspartner)
df_portal_physisch_empty_email = find_portal_vs_physisch_doublette(df_organisationen, organisationen=True, strict_email=False, only_Geschaeftspartner=only_with_Geschaeftspartner)

df_portal_physisch_only_nonempty_email = add_organisationsrollen_string_columns(df_portal_physisch_only_nonempty_email, df_organisationsrollen)
df_portal_physisch_empty_email = add_organisationsrollen_string_columns(df_portal_physisch_empty_email, df_organisationsrollen)

cols_to_keep=["ReferenceID", "Name", "Objekt_link", "address_full", "VerknuepftesObjekt", "VerknuepftesObjektID", "EMailAdresse", "Versandart", "Geschaeftspartner", "Servicerole_string", "Organisationsrollen", "Organisationrollen_ProduktID", "cluster_id", "score", "master", "masterID"]
df_portal_physisch_only_nonempty_email = final_touch(df_portal_physisch_only_nonempty_email, cols_to_keep)
df_portal_physisch_empty_email = final_touch(df_portal_physisch_empty_email, cols_to_keep)

with pd.ExcelWriter('output/Organisationen_Portal_Vs_Physisch.xlsx', engine='openpyxl') as writer:
    df_portal_physisch_only_nonempty_email.to_excel(writer, sheet_name='nonempty_email_only', index=False)
    df_portal_physisch_empty_email.to_excel(writer, sheet_name='empty_email_allowed', index=False)
    